<div style="background:#2b6684">
</div>
           

### Song Popularity Prediction || EDA 1
1. 
2. 

<div style="background:#2b4444;font-family:'Times';font-size:35px;color:  #F0CB8E" >&ensp;Song Popularity Prediction || EDA1  
</div>
<div class="alert alert-warning" role="alert">
<ul style="font-family:cursive;font-size:18px" >
    
<li><a href = "#1"> 1. Introduction </a></li>     
<li ><a href = "#2"> 2. Preparations </a></li> 
<ul>
<li ><a  href = "#200"> 2.1 Load Libraries </a></li>
<li ><a  href = "#201"> 2.2 Load Data </a></li>
</ul>
<li><a href = "#3"> 3. Overview: structure and data content</a></li>
<ul> 
<li  ><a href = "#300">3.1 Look at the data</a></li>
<li  ><a href = "#301">3.2 Missing Values</a></li>
</ul>  
<li ><a  href = "#4"> 4. Visualisation - Individual features </a></li>
<ul> 
<li  ><a href = "#400">4.1 Predictor features</a></li>
<li  ><a href = "#401">4.2 Target: Song Popularity</a></li>
</ul>  

<li ><a  href = "#5"> 5. Feature interactions</a></li>
<ul>
<li ><a  href = "#500"> 5.1 Target impact</a></li>
<li ><a  href = "#501"> 5.2 Feature-Feature interaction (correlation)</a></li>
<li ><a  href = "#502"> 5.3 Feature-target interactions</a></li>
    </ul></ul>
</div>


[<p style="font-family:cursive;font-size:18px; color:#A20404">back to Table of Contents</p>](#table-of-contents)
<a id="1"></a>

<div style="background:#2b6684   ;font-family:'Times';font-size:35px;color:  #F0CB8E" >&ensp;Introduction</div>
<div class="alert alert-warning" role="alert">
<ul style="font-family:cursive;font-size:18px; color:#A20404" >
<li>Welcome to one of the first Kaggle Community Competitions in the world!</li>
<li>This challenge is about predicting Song Popularity based on a set of different features.</li>
<li>This specific competition is about predicting Song Popularity based on a set of different features. It is positioned as the first in a series of starter competitions that provide a gentle introduction to machine learning (ML) and data science (DS). This is a classification challenge, with the evaluation metric being chosen as <a style="color:  blue" href="https://en.wikipedia.org/wiki/Receiver_operating_characteristic" target="_blank"><u>AUC</u></a></li>
<li>The data consists of the standard Kaggle train.csv and test.csv files, with a sample_submission.csv to show you the structure of the file that should be submitted.</li>
<li>Checkout more about Data: <a style="color:  blue" href="https://www.kaggle.com/c/song-popularity-prediction/discussion/301616" target="_blank"> <u>Data Dictionary</u></a> by <a style="color:  blue" href="https://www.kaggle.com/remekkinas" target="_blank"> <u>Remek Kinas</u></a></li>   
   
<li>This notebook started out as a live-coding session on Abhishek Thakur’s channel on Youtube:<a style="color:  blue" href="https://www.youtube.com/watch?v=JXF-7rCcR1c" target="_blank"> <u>Link to the Youtube Video</u></a> </li>

<div style 

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import optuna
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb

In [3]:
df_train = pd.read_csv('../input/song-popularity-prediction/train.csv', index_col = 'id')
df_test = pd.read_csv('../input/song-popularity-prediction/test.csv', index_col = 'id')
df_train.shape, df_test.shape

In [4]:
df_train.head(5)

In [5]:
df_train.info()

In [6]:
df_train.isnull().sum()

In [7]:
sns.countplot(x='song_popularity', data = df_train)

In [8]:
categorical_cols = ['key', 'audio_mode', 'time_signature']


### missing values
Let's combine the test and train data to see the distribution of missing values in the whole dateset. 

In [9]:
#combine train and test data together to deal with missing values
df_train['isTrain'] = True
df_test['isTrain'] = False
tt = pd.concat([df_train, df_test]).reset_index(drop=True).copy()
df_train.shape, df_test.shape, tt.shape

In [10]:
tt.head()

In [11]:
tt.tail()

In [12]:
list(tt.columns)

Before we start, we should ask why there is missing values in the data. 
examples could be,
1. Sensor data where the sensor went offline
2. Survey data where some answeres were not answered
3. A kaggle competition where the host wants to make the problem hard
Sometimes values being null itself is a indicator to help out with the modeling technique. It could be the values are radomly missing or the fact that they missing could be a feature itself. 

Let's see the missing value counts in train vs. test

In [13]:
df_train.isna()

In [14]:
df_test.isna()

In [15]:
df_train.isna().sum()

In [16]:
df_test.isna().sum()

We see in both train and test data, there are some features with missing values and some without. 

Now we want to see is there any diffirenct between the null values in the trian and test data in the dataset

In [17]:
#creating a dataframe with null values in the test and train dataset
ncounts = pd.DataFrame([df_train.isna().mean(), df_test.isna().mean()]).T
ncounts = ncounts.rename(columns={0: 'train_missing', 1: 'test_missing'})
ncounts

In [18]:
# filter out the non-missing features and 
# plot the features with missing vlaues only
ncounts.query('train_missing > 0').plot(kind='barh',
                                        figsize=(8,5), 
                                        title='%of missing values')

This is very interesting that all the features in both in trian and test data, approximately 10% of the data is missing. This is worth more exploration and thinking



In [33]:
nacols = ['song_duration_ms',
         'acousticness',
         'danceability',
         'energy',
         'instrumentalness',
         'key',
         'liveness',
         'loudness']

### How many missing values per observations/example

In [32]:
# counts of null per row/example
df_train.isna().sum(axis=1)

In [34]:
tt['n_missing'] = tt[nacols].isna().sum(axis=1)
df_train['n_missing'] = df_train[nacols].isna().sum(axis=1)
df_test['n_missing'] = df_test[nacols].isna().sum(axis=1)

In [37]:
tt['n_missing']

In [36]:
# how many of the samples has 0 misisng values, 1 missing values and so on
tt['n_missing'].value_counts()

we see that most of our samples has no missing values, 18880 samples has 1 missing values, 7421 samples has 2 missing values and so on. This make sence since only 10% of the data is missing in this dataset

In [38]:
tt['n_missing'].value_counts().plot(kind='bar',
                                   title='Number of Missing Valeus per Sample')

In [39]:
tt.query('n_missing == 6') # samples with 6 missing values

We see two of these sample are from training data and one from test data. There is actually real sparcity of features that could be used to predict the song popularity of these examples.Imputation will take a real big toll in these examples in particular

### Is there an imbalance in missing values when splitting by other features?
One thing we can check that we have some categorical features and we have our target feature. Are there any imbalance in missing values between these grouping? Let's look at the binary feature 'audio mode'.

In [42]:
tt['audio_mode'].value_counts()

In [45]:
cat_features = ['key', 'audio_mode']
tt.groupby('audio_mode')['n_missing'].mean()

we about 80% in audio mode 0 and about 80% in audio mode 1 has missing values.which is pretty even. Now let's look at time signature

In [46]:
tt.groupby('time_signature')['n_missing'].mean()

Again pretty similar. Might be little bit of deviation here. Let's see with aggregated mean with count

In [47]:
tt.groupby('time_signature')['n_missing'].agg(['mean', 'count'])

The mean value might be lower but we see there so diffirent with the count with time signature 2 and 5.

### Prep - Create tag columns with missing indicators

In [58]:
# creating a dataframe with na cols as binary true or false
tt_missing_tag_df = tt[nacols].isna() 
#  feature name + missing added with it
tt_missing_tag_df.columns = [f"{c}_missing" for c in tt_missing_tag_df]

In [59]:
tt_missing_tag_df.head()

In [61]:
list(tt_missing_tag_df.columns)

In [60]:
#concat these dataframe with the existing dataframe
tt= pd.concat([tt, tt_missing_tag_df], axis =1)
tt.head()

In [64]:
tt.columns

### Protip: Try to predict the target using only missing value indicators as features

In [67]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score

lr = LogisticRegressionCV(scoring='accuracy')
X = tt.query('isTrain')[
    ['song_duration_ms_missing',
     'acousticness_missing',
     'danceability_missing',
     'energy_missing',
     'instrumentalness_missing',
     'key_missing',
     'liveness_missing',
     'loudness_missing']
]
y = tt.query('isTrain')['song_popularity']
lr.fit(X, y)
lr.socre(X, y)
preds = lr.predict_proba(X)[:, 0]
roc_auc_score(y, preds)